In [1]:
import pandas as pd

def read_xlsx(file):
    df = pd.read_excel(file, header=None)
    return df[0].tolist()

def match_lists(list1, list2):
    matches = []
    list1_to_remove = []
    
    for i in list1:
        if i in list2:
            matches.append((i, i))
            list1_to_remove.append(i)
            list2.remove(i)
    
    for i in list1_to_remove:
        list1.remove(i)
    return matches

def find_combination(target, numbers):
    if target == 0:
        return []
    if target < 0 or not numbers:
        return None
    for i, num in enumerate(numbers):
        remaining = numbers[i + 1:]
        result = find_combination(target - num, remaining)
        if result is not None:
            return [num] + result
    return None

def match_sums(list1, list2):
    matches = []
    unmatches = []
    list1.sort()
    list2.sort(reverse=True)

    group_counter = 65  # ASCII value for 'A'
    used_numbers = set()  # To keep track of used numbers

    unmatched_values = []

    while list1:
        x = list1.pop(0)
        if x in used_numbers:
            continue
        combination = find_combination(x, list2)
        if combination:
            group_letter = chr(group_counter)
            matches.append((x, combination, group_letter))
            group_counter += 1
            used_numbers.add(x)
            for num in combination:
                list2.remove(num)
        else:
            unmatched_values.append(x)

    # Add unmatched values to unmatches list
    for value in unmatched_values:
        unmatches.append((value, '', '0'))

    # Add unmatched values from list2
    for num in list2:
        unmatches.append(('', num, '0'))

    return matches, unmatches

def save_to_excel(exact_matches, unmatched_values, remaining_matches):
    # Create DataFrame for exact matches
    df_exact = pd.DataFrame(exact_matches, columns=['bank', 'bc'])

    # Prepare DataFrame for unmatched values and remaining matches
    combined_data = []

    # Add unmatched bank values
    for bank_value, _, _ in unmatched_values:
        if bank_value != '':
            combined_data.append({'bank': -bank_value, 'bc': None, 'group': '0'})
    
    # Add unmatched bc values
    for _, bc_value, _ in unmatched_values:
        if bc_value != '':
            combined_data.append({'bank': None, 'bc': bc_value, 'group': '0'})
    
    # Add remaining matches
    for bank_value, bc_values, group_label in remaining_matches:
        combined_data.append({'bank': -bank_value, 'bc': None, 'group': group_label})
        for bc_value in bc_values:
            combined_data.append({'bank': None, 'bc': bc_value, 'group': group_label})

    # Create DataFrame for combined unmatched and remaining matches
    df_combined = pd.DataFrame(combined_data)

    # Save to Excel with only 2 sheets
    with pd.ExcelWriter('poprawa_output.xlsx') as writer:
        df_exact.to_excel(writer, sheet_name='Exact Matches', index=False)
        df_combined.to_excel(writer, sheet_name='Unmatched and Remaining Matches', index=False)

    print("poprawa_output.xlsx has been created.")
    

In [2]:
file1 = 'bank.xlsx'
file2 = 'bc.xlsx'

In [3]:
list1 = read_xlsx(file1)
list2 = read_xlsx(file2)

# Process matches
exact_matches = match_lists(list1, list2)
remaining_matches, unmatched_values = match_sums(list1, list2)

save_to_excel(exact_matches, unmatched_values, remaining_matches)

poprawa_output.xlsx has been created.
